In [ ]:
!pip install nnunetv2 --quiet
!pip install pydicom --quiet
!pip install boxsdk --quiet
!pip install pynrrd --quiet

import nnunetv2
import numpy as np
import pydicom
import zipfile
import re
import gzip
import matplotlib.pyplot as plt
import pandas as pd
import nrrd
import boxsdk
from boxsdk import Client, OAuth2
import os

In [ ]:
runtime_dir = '/content/runtime_files/'
files_dir = '/content/runtime_files/data_dir'
data_dir = '/content/runtime_files/data_dir/nnUNet_raw'


if not os.path.exists(runtime_dir):
    os.mkdir(runtime_dir)
if not os.path.exists(files_dir):
    os.mkdir(files_dir)
if not os.path.exists(data_dir):
    os.mkdir(data_dir)


In [ ]:
input = """

class nnUNetTrainerDA5_200epochs(nnUNetTrainerDA5):
    def __init__(self, plans: dict, configuration: str, fold: int, dataset_json: dict,
                 device: torch.device = torch.device('cuda')):
        super().__init__(plans, configuration, fold, dataset_json, device)
        self.num_epochs = 200

"""

da_file = "/usr/local/lib/python3.11/dist-packages/nnunetv2/training/nnUNetTrainer/variants/data_augmentation/nnUNetTrainerDA5.py"
with open(da_file, 'a') as file:
    file.write(input)

In [ ]:
# SET TRIAL VARIABLES

trial_num = 11
trail_iteration= "trial"+str(trial_num)

results_dir = '/content/drive/MyDrive/bsd/07_oropharyngeal_tumor_segmentation/20250324/results/nnUNET'
trial_path = os.path.join(results_dir, trail_iteration)
results_path = os.path.join(trial_path, "nnUNet_results")
preprocessed_path = os.path.join(trial_path, "nnUNet_preprocessed")

if not os.path.exists(trial_path):
  os.makedirs(trial_path)
if not os.path.exists(results_path):
  os.makedirs(results_path)
if not os.path.exists(preprocessed_path):
  os.makedirs(preprocessed_path)

In [ ]:
# Set environment variable
os.environ['nnUNet_raw'] = data_dir
os.environ['nnUNet_preprocessed'] = preprocessed_path
os.environ['nnUNet_results'] = results_path

print(os.environ.get('nnUNet_raw'))
print(os.environ.get('nnUNet_preprocessed'))
print(os.environ.get('nnUNet_results'))

In [ ]:
#!nnUNetv2_plan_and_preprocess -d 555 556 --verify_dataset_integrity
#!nnUNetv2_plan_and_preprocess -d 557 --verify_dataset_integrity

In [ ]:
# Manual replace 557 CV splits to align with 555, 556

In [ ]:
#datasets = ['555', '556', '557']
configs = ['3d_fullres'] # ['3d_fullres', '2d']
folds = ['0', '1', '2', '3', '4']
trainer = "nnUNetTrainerDA5_200epochs"

for fold in folds:
    for dataset in datasets:
        for config in configs:
            cmd = f"nnUNetv2_train {dataset} {config} {fold} -tr {trainer} --npz"
            print(cmd)
            !{cmd}